### Обучить полносвязную модель на MNIST

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

input_d = 28*28
hidden_d = 100
n_classes = 10
n_epoch = 6
batch_size = 100
lr = 0.001

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# loading MNIST
train_data = torchvision.datasets.MNIST(root='./dataset', train=True, transform=transforms.ToTensor(), download=True)
test_data = torchvision.datasets.MNIST(root='./dataset', train=False, transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

class Model(nn.Module):
    def __init__(self, input_d, hidden_d, n_classes):
        super(Model, self).__init__()
        self.l1 = nn.Linear(input_d, hidden_d)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_d, n_classes)
        
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        return out 
    
model = Model(input_d, hidden_d, n_classes)

criterion = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(), lr=lr)


n_total_steps = len(train_loader)
for epoch in range(n_epoch):
    for step, (images, labels) in enumerate(train_loader):
        
        images = images.reshape(-1, input_d).to(device)
        labels = labels.to(device)
        
        output = model(images)
        loss = criterion(output, labels)
        
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        
        if (i+1) % 100 == 0:
            print(f'epoch {epoch+1} / {num_epochs}, step {step+1} / {n_total_steps}, loss = {loss.item():.3f}')

            
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(n_classes)]
    n_class_samples = [0 for i in range(n_classes)]
    
    for images, labels in test_loader:
        images = images.reshape(-1, input_d).to(device)
        labels = labels.to(device)
        output = model(images)
        
        _, predictions = torch.max(output, 1)
        n_samples += labels.shape[0]
        n_correct += (predictions == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            if (label == predictions[i]):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
        
    acc = 100.0 * n_correct / n_samples
    print(f'accuracy = {acc} %\n')
    
    for i in range(n_classes):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'accuracy of class {i}(num = {n_class_samples[i]}): {acc:.2f} %')

accuracy = 97.27 %

accuracy of class 0(num = 980): 97.24 %
accuracy of class 1(num = 1135): 99.30 %
accuracy of class 2(num = 1032): 95.74 %
accuracy of class 3(num = 1010): 98.42 %
accuracy of class 4(num = 982): 98.47 %
accuracy of class 5(num = 892): 94.51 %
accuracy of class 6(num = 958): 97.60 %
accuracy of class 7(num = 1028): 98.05 %
accuracy of class 8(num = 974): 95.59 %
accuracy of class 9(num = 1009): 97.22 %


### Обучить глубокую сверточную сеть на MNIST

In [22]:
n_epoch = 4
batch_size = 4
lr = 0.001
n_classes = 10

# loading MNIST
train_data = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_data = torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

class ConvModel(nn.Module):
    def __init__(self):
        super(ConvModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=5, kernel_size=3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(in_channels=5, out_channels=16, kernel_size=5)
        self.fc1 = nn.Linear(16*4*4, 100)
        self.fc2 = nn.Linear(100, 80)
        self.fc3 = nn.Linear(80, 10)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*4*4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = ConvModel().to(device)

criterion = nn.CrossEntropyLoss()
optimiser = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        
        if (i+1) % 1000 == 0:
            print(f'epoch {epoch+1} / {num_epochs}, step {i+1} / {n_total_steps}, loss = {loss.item():.4f}')

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(n_classes)]
    n_class_samples = [0 for i in range(n_classes)]
    
    for images, labels in test_loader:
        labels = labels.to(device)
        outputs = model(images)
        
        _, predictions = torch.max(outputs, 1)
        n_samples += labels.shape[0]
        n_correct += (predictions == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predictions[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
        
    acc = 100.0 * n_correct / n_samples
    print(f'accuracy = {acc} %\n')
    
    for i in range(n_classes):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'accuracy of {i}: {acc:.2f} %')

epoch 1 / 4, step 1000 / 15000, loss = 2.2924
epoch 1 / 4, step 2000 / 15000, loss = 2.3288
epoch 1 / 4, step 3000 / 15000, loss = 2.2878
epoch 1 / 4, step 4000 / 15000, loss = 2.2954
epoch 1 / 4, step 5000 / 15000, loss = 2.3274
epoch 1 / 4, step 6000 / 15000, loss = 2.2879
epoch 1 / 4, step 7000 / 15000, loss = 2.2530
epoch 1 / 4, step 8000 / 15000, loss = 2.2882
epoch 1 / 4, step 9000 / 15000, loss = 2.2942
epoch 1 / 4, step 10000 / 15000, loss = 2.3076
epoch 1 / 4, step 11000 / 15000, loss = 2.2773
epoch 1 / 4, step 12000 / 15000, loss = 2.2628
epoch 1 / 4, step 13000 / 15000, loss = 2.2873
epoch 1 / 4, step 14000 / 15000, loss = 2.2797
epoch 1 / 4, step 15000 / 15000, loss = 2.3070
epoch 2 / 4, step 1000 / 15000, loss = 2.2139
epoch 2 / 4, step 2000 / 15000, loss = 2.1152
epoch 2 / 4, step 3000 / 15000, loss = 1.8232
epoch 2 / 4, step 4000 / 15000, loss = 1.9579
epoch 2 / 4, step 5000 / 15000, loss = 0.6978
epoch 2 / 4, step 6000 / 15000, loss = 0.2400
epoch 2 / 4, step 7000 / 150